From https://datascience.stackexchange.com/questions/55566/tool-for-labeling-audio

In [1]:
from pathlib import Path
import pandas as pd
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm


In [24]:
# build dataset pandas df
train_dataset = pd.read_pickle('saved_features_train.pickle')
test_dataset = pd.read_pickle('saved_features_test.pickle')

print('Finished feature extraction from ', len(train_dataset), ' files')

Finished feature extraction from  4728  files


In [26]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
x_train = np.nan_to_num(np.array(train_dataset.feature.tolist()))
y_train = np.array(train_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train)) 

x_test = np.nan_to_num(np.array(test_dataset.feature.tolist()))
y_test = np.array(test_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_test = to_categorical(le.fit_transform(y_test)) 

In [29]:
import importlib
import simple_model

importlib.reload(simple_model)
model = simple_model.create_model(256)

In [30]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                

In [32]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
saved_folder = 'model_saves'
if not os.path.isdir(saved_folder):
    os.mkdir(saved_folder)

num_epochs = 100
num_batch_size = 256
model_name = 'weights.best.simple_cnn_more_features.hdf5'
checkpointer = ModelCheckpoint(filepath=f'{saved_folder}/{model_name}', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 4728 samples, validate on 856 samples
Epoch 1/100
4728/4728 [==============================] - 0s 24us/step - loss: 0.0212 - accuracy: 0.9918 - val_loss: 0.0675 - val_accuracy: 0.9743

Epoch 00001: val_loss improved from inf to 0.06750, saving model to model_saves/weights.best.simple_cnn_more_features.hdf5
Epoch 2/100
4728/4728 [==============================] - 0s 29us/step - loss: 0.0260 - accuracy: 0.9932 - val_loss: 0.0625 - val_accuracy: 0.9755

Epoch 00002: val_loss improved from 0.06750 to 0.06249, saving model to model_saves/weights.best.simple_cnn_more_features.hdf5
Epoch 3/100
4728/4728 [==============================] - 0s 34us/step - loss: 0.0204 - accuracy: 0.9920 - val_loss: 0.0631 - val_accuracy: 0.9755

Epoch 00003: val_loss did not improve from 0.06249
Epoch 4/100
4728/4728 [==============================] - 0s 41us/step - loss: 0.0257 - accuracy: 0.9905 - val_loss: 0.0637 - val_accuracy: 0.9755

Epoch 00004: val_loss did not improve from 0.06249
Epoch 5/100
4

In [33]:
# Evaluating the model on the training and testing set
model.load_weights(f"model_saves\{model_name}")
score = model.evaluate(x_train, y_train, verbose=0)
print(f"Training Accuracy: {score[1]*100:.1f}%")

score = model.evaluate(x_test, y_test, verbose=0)
print(f"Testing Accuracy: {score[1]*100:.1f}%")

Training Accuracy: 99.8%
Testing Accuracy: 98.0%
